In [1]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('max_colwidth', 5000)

df = pd.read_csv('DB/Original_data.csv', index_col=[0])
df = df.sample(frac=0.5)


In [2]:
#Check if there are null values 
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

In [3]:
#create column with answers aggregate by title of the questions
df2 = df 
func = lambda x: "\n".join(x)
df2 = df2.groupby('id')["answers"].agg([("answers",func)])



In [4]:
# concat answers for each post beacause there are a record with question duplicated for each answer to that question 


grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg("sum","score")

grouped_df = pd.DataFrame(grouped)
grouped_df = pd.merge(grouped_df, df2, left_on='id', right_on='id', how='left')
grouped_df.head()



id  \
0   742   
1   773   
2  1829   
3  2933   
4  5102   

                                                                                       title  \
0                                                                      Class views in Django   
1                                                          How do I use itertools.groupby()?   
2  How do I make a menu that does not require the user to press [enter] to make a selection?   
3                           Create a directly-executable cross-platform GUI app using Python   
4                                    How do you set up Python scripts to work in Apache 2.0?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 body  \
0                                                                                                                                      <p><a href="http://www.djangoproject.com/" rel="noreferrer">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\n\n<p>For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc...</p>\n\n<p>One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea? </p>   
1  <p>I haven't been able to find an understandable explanation of how to actually use Python's <code>itertools.groupby()</code> function.  What I'm trying to do is this:</p>\n<ul>\n<li>Take a list - in this case, the children of an objectified <code>lxml</code> element</li>\n<li>Divide it into groups based on some criteria</li>\n<li>Then later iterate over each of these groups separately.</li>\n</ul>\n<p>I've reviewed <a href="https://docs.python.org/3/library/itertools.html#itertools.groupby" rel="noreferrer">the documentation</a>, but I've had trouble trying to apply them beyond a simple list of numbers.</p>\n<p>So, how do I use of <code>itertools.groupby()</code>?  Is there another technique I should be using?  Pointers to good &quot;prerequisite&quot; reading would also be appreciated.</p>   
2                                                           <p>I've got a menu in Python. That part was easy. I'm using <code>raw_input()</code> to get the selection from the user. </p>\n\n<p>The problem is that <code>raw_input</code> (and input) require the user to press <kbd>Enter</kbd> after they make a selection. Is there any way to make the program act immediately upon a keystroke? Here's what I've got so far:</p>\n\n<pre><code>import sys\nprint """Menu\n1) Say Foo\n2) Say Bar"""\nanswer = raw_input("Make a selection&gt; ")\n\nif "1" in answer: print "foo"\nelif "2" in answer: print "bar"\n</code></pre>\n\n<p>It would be great to have something like</p>\n\n<pre><code>print menu\nwhile lastKey = "":\n    lastKey = check_for_recent_keystrokes()\nif "1" in lastKey: #do stuff...\n</code></pre>   
3                                                                                                                            

In [5]:
#Preprocessing Part
#Answers
#Removing tags
import preprocessing

answers = grouped_df["answers"]
preprocessing.remove_tags(answers)



#Clearing text 
answers_processed = answers.apply(lambda x: preprocessing.clear_text(x,"w2v"))


In [6]:
answers_processed.head()
answers_processed.isna().sum() 

0

In [7]:
#Questions
#Merge title with body 
questions = grouped_df["title"] + grouped_df["body"]
preprocessing.remove_tags(questions)
questions

0                                                                 Class views in Django  view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach    For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc    One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?  
1                                                                              How do I use itertoolsgroupby()? I haven't been able to find an understandable explanation of how to actually use Python's  function  What I'm trying to do is this:     Take a list - in this case, the children of an objectified  element   Divide it into gro

In [8]:
#Clearing text 
questions_processed = questions.apply(lambda x: preprocessing.clear_text(x,"w2v"))
questions_processed

0                   class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea 
1                                     use itertoolsgroupby nt able find understandable explanation actually use python function trying take list case children objectified element divide groups based criteria later iterate groups separately reviewed trouble trying apply beyond simple list numbers use another technique using pointers good quot prerequisite quot reading would also appreciated 
2                                                                                                                                                          make menu require user press enter make selection got men

In [9]:
#Create a column to add in db with question title, question body and all the answers processed, to train later the W2V embeddings
post_corpus = questions_processed + answers_processed
grouped_df.drop("answers", axis=1, inplace=True)
grouped_df.drop("body", axis=1, inplace=True)
grouped_df["post_corpus"] = post_corpus
grouped_df["questions"] = questions
grouped_df

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,python|django|views|oop,28,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc sounds like trying combine things nt combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan use django generic views easily achieve desired functionality thorough django generic views needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate call view class works checking first argument must actual http request would nonsense attept instantiate view class instance posted snippet always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?"
1,773,How do I use itertools.groupby()?,python|itertools,300,use itertoolsgroupby nt able find understandable explanation actually use python function trying take list case children objectified element divide groups based criteria later iterate groups separately reviewed trouble trying apply beyond simple list numbers use another technique using pointers good quot prerequisite quot reading would also appreciated use groupby group things iterate give groupby iterable optional key functioncallable check items come iterable returns iterator gives twotuple result key callable actual items another iterable help example groupby using coroutine group count uses key callable case spit count however many iterations grouped subiterator elements prints would like give another example groupby without sort working adapted example james sulak output two groups vehicule whereas one could expect one group another example results note igroup iterator subiterator documentation calls useful chunking generator another example groupby keys sorted following example items xx grouped values yy case one set zeros output first followed set ones followed set zeros produces one useful example came across may helpful sample input sample output basic implementation helped understand function hope helps others well neato trick groupby run length encoding one line give list first element char number repetitions edit note separates sql semantics itertools nt general ca nt sort iterator advance groups key nt merged captsolo tried example nt work output see two two e got separate groups realized need sort list passed groupby function correct usage would output remembering list sorted groupby function

In [10]:
#Filter out the tags, selecting only the 20 most common for better processing 
# Convert raw text data of tags into lists
grouped_df["tags"] = grouped_df["tags"].apply(lambda x: x.split('|'))   

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}
for tags in grouped_df["tags"]:
    for tag in tags:
        if "python" not in tag and "python-2.7" not in tag and "python-3.x" not in tag:
            
            if tag not in tag_freq_dict:
                tag_freq_dict[tag] = 0
            else:
                tag_freq_dict[tag] += 1
        else:
            tags.remove(tag)
            

grouped_df["tags"]


0                                              [django, views, oop]
1                                                       [itertools]
2                                                                []
3         [user-interface, deployment, tkinter, release-management]
4                                                 [apache, apache2]
                                    ...                            
185173                                           [pandas, for-loop]
185174                                                      [numpy]
185175             [machine-learning, graph, pytorch, data-science]
185176                                                           []
185177                  [if-statement, dutch-national-flag-problem]
Name: tags, Length: 185178, dtype: object

In [11]:
#Selecting the most common number of tags in our database 
import heapq
most_common_tags = heapq.nlargest(20, tag_freq_dict, key=tag_freq_dict.get)
most_common_tags

['dataframe',
 'numpy',
 'pandas',
 'dictionary',
 'django',
 'list',
 'beautifulsoup',
 'matplotlib',
 'tkinter',
 'flask',
 'csv',
 'keras',
 'javascript',
 'django-models',
 'web-scraping',
 'for-loop',
 'scipy',
 'scikit-learn',
 'datetime',
 'string']

In [12]:
#Select only the data with at least one of the most common tags 
final_indices = []
for i,tags in enumerate(grouped_df["tags"].values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>0:   # The minimum length for common tags should be 1
        final_indices.append(i)

final_data = grouped_df.iloc[final_indices]

final_data 

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,"[django, views, oop]",28,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc sounds like trying combine things nt combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan use django generic views easily achieve desired functionality thorough django generic views needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate call view class works checking first argument must actual http request would nonsense attept instantiate view class instance posted snippet always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?"
3,2933,Create a directly-executable cross-platform GUI app using Python,"[user-interface, deployment, tkinter, release-management]",26,create directlyexecutable crossplatform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line gui pretty much reference another way develop rapid user interface write web app run locally display app browser plus go tkinter option suggested lubos hasko may want try portablepy app run windows environment without python since python installed nearly every nonwindows os default thing really need make sure nonstandard libraries use installed said possible build executables include python interpreter libraries use likely create large executable however macos x even includes support xcode ide creating full standalone gui apps run user running os x use example bring one folder files needed run app even user python pc talking windows apple os need executable file think come python without need installing create file create setuppy code save folder put program folder setuppy put folder program want make distribuitable es ps change name file anything want example run cmd folder folder right click shift choose start cmd write cmd gt python setuppy dist folder files need zip distribute pyinstaller install cmd run cmd folder file update read post make exe windows pyinstaller proper way one file images also basically python wrapper gnome toolkit found easier wrap mind around tkinter coming pretty much knowledge gui pr

In [13]:
#Create a column only for the processed title of the questions 
final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x,"w2v"))

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_11944/2150014301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x,"w2v"))


In [14]:
# Normalize numeric data for the scores
final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_11944/3151826978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())


In [15]:
# Combine the lists back into text data
final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))

final_data

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_11944/3494508092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))


,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.009946,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc sounds like trying combine things nt combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan use django generic views easily achieve desired functionality thorough django generic views needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate call view class works checking first argument must actual http request would nonsense attept instantiate view class instance posted snippet always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?",class views django
3,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.009108,create directlyexecutable crossplatform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line gui pretty much reference another way develop rapid user interface write web app run locally display app browser plus go tkinter option suggested lubos hasko may want try portablepy app run windows environment without python since python installed nearly every nonwindows os default thing really need make sure nonstandard libraries use installed said possible build executables include python interpreter libraries use likely create large executable however macos x even includes support xcode ide creating full standalone gui apps run user running os x use example bring one folder files needed run app even user python pc talking windows apple os need executable file think come python without need installing create file create setuppy code save folder put program folder setuppy put folder program want make distribuitable es ps change name file anything want example run cmd folder folder right click shift choose start cmd write cmd gt python setuppy dist folder files need zip distribute pyinstaller install cmd run cmd folder file update read post make exe windows pyinstaller proper way one file images also basically python wrapper gnome toolkit found easier wrap mind around tkinter c

In [16]:
#Check if the final data has some null values 
final_data.isna().sum()

final_data.dropna()
#final_data = final_data[final_data['post_corpus'].notna()]

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.009946,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc sounds like trying combine things nt combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan use django generic views easily achieve desired functionality thorough django generic views needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate call view class works checking first argument must actual http request would nonsense attept instantiate view class instance posted snippet always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?",class views django
3,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.009108,create directlyexecutable crossplatform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line gui pretty much reference another way develop rapid user interface write web app run locally display app browser plus go tkinter option suggested lubos hasko may want try portablepy app run windows environment without python since python installed nearly every nonwindows os default thing really need make sure nonstandard libraries use installed said possible build executables include python interpreter libraries use likely create large executable however macos x even includes support xcode ide creating full standalone gui apps run user running os x use example bring one folder files needed run app even user python pc talking windows apple os need executable file think come python without need installing create file create setuppy code save folder put program folder setuppy put folder program want make distribuitable es ps change name file anything want example run cmd folder folder right click shift choose start cmd write cmd gt python setuppy dist folder files need zip distribute pyinstaller install cmd run cmd folder file update read post make exe windows pyinstaller proper way one file images also basically python wrapper gnome toolkit found easier wrap mind around tkinter c

In [17]:
# Save the data
final_data.to_csv('DB/Preprocessed_data.csv', index=False)